In [1]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
from collections import defaultdict

### change directory to import more scripts
sys.path.append(str(Path().resolve().parents[0]))

from utils import tools
import generate_wind

if os.getcwd().split(os.sep)[-1] == 'evaluation':
    os.chdir('..')

In [4]:
config = tools.load_config("config.yaml")

val_dir = config['data']['re_ninja_dir']
synth_dir = os.path.join(config['data']['synth_dir'], 'wind', 'no_density', 'wind_noage')
turbine_dir = config['data']['turbine_dir']
turbine_power = config['data']['turbine_power']
turbine_path = os.path.join(turbine_dir, turbine_power)
specs_path = config['data']['turbine_specs']
specs_path = os.path.join(turbine_dir, specs_path)
cp_path = config["data"]["turbine_cp"]
cp_path = os.path.join(turbine_dir, cp_path)
results_dir = 'results'

os.makedirs(results_dir, exist_ok=True)

features = config['features']

params = config['wind_params']

mapping = {'power_t1': 'Enercon E-70 E4 2.300',
           'power_t2': 'Enercon E-82 E2 2.000',
           'power_t3': 'Enercon E-115 2.500',
           'power_t4': 'Vestas V90',
           'power_t5': 'Vestas V112-3.45',
           'power_t6': 'Vestas V80-1.8'
           }

turbine_mapping = {'electricity_1': 'power_t1',
                    'electricity_2': 'power_t2',
                    'electricity_3': 'power_t3',
                    'electricity_4': 'power_t4',
                    'electricity_5': 'power_t5',
                    'electricity_6': 'power_t6'
                    }

In [5]:
# get validationd data
files = os.listdir(val_dir)
station_files = [f for f in files if 'metadata' not in f and f.endswith('.csv')]
metadata_files = [f for f in files if 'metadata' in f and f.endswith('.csv')]
station_files.sort()
metadata_files.sort()
print(len(station_files), 'station files in validation data found')

files = os.listdir(synth_dir)
station_files_synthetic = [f for f in files if 'parameter' not in f and f.endswith('.csv')]
station_files_synthetic.sort()
print(len(station_files_synthetic), 'station files in synthetic data found')
turbine_params = pd.read_csv(os.path.join(synth_dir, 'turbine_parameter.csv'), sep=';')
wind_params = pd.read_csv(os.path.join(synth_dir, 'wind_parameter.csv'), sep=';', dtype={"park_id": str})

raw_power_curves, cp_curves, specs = generate_wind.get_turbines(turbine_path=turbine_path,
                                                        cp_path=cp_path,
                                                        specs_path=specs_path,
                                                        params=params)
power_curves = pd.DataFrame()
for turbine in raw_power_curves.columns:
    power_curve = generate_wind.interpolate(power_curve=raw_power_curves[turbine],
                                            cut_out=specs[turbine]['cut_out'])
    power_curves[turbine] = power_curve
power_curves /= 1000 # convert to kW

39 station files in validation data found
110 station files in synthetic data found


In [6]:
# filter the list
filtered_list = [
    synth for synth in station_files_synthetic
    if any(synth.split('_')[1].split('.')[0] in station for station in station_files)
]
station_files_synthetic = filtered_list

In [7]:
val_dfs = []
dfs = []

for st, md in zip(station_files, metadata_files):
    station = pd.read_csv(os.path.join(val_dir, st))
    station['datetime'] = pd.to_datetime(station['datetime'], utc=True)
    station.set_index('datetime', inplace=True)
    val_dfs.append(station)
metadata = pd.read_csv(os.path.join(val_dir, md))

for st in station_files_synthetic:
    station = pd.read_csv(os.path.join(synth_dir, st), sep=';')
    station['timestamp'] = pd.to_datetime(station['timestamp'], utc=True)
    station.set_index('timestamp', inplace=True)
    station = station['2024-01-01':'2024-12-31']
    for key, value in turbine_mapping.items():
        station[value] /= 1000 # convert to kW
    #metadata = pd.read_csv(os.path.join(path, md))
    dfs.append(station)

In [8]:
data_a, data_b = dfs[0], val_dfs[0]
for df, val_df in zip(dfs[1:], val_dfs[1:]):
    data_a = pd.concat([data_a, df], axis=0)
    data_b = pd.concat([data_b, val_df], axis=0)

In [9]:
def get_stats_and_diffs(data_a, data_b, mapping, turbine_mapping, print_stats=True):
    stats = {}
    power_diffs = {}
    wind_diffs = {}
    synth_sum = 0
    re_ninja_sum = 0
    for i, turbine in enumerate(map(str, range(1, 7))):
        synthetic_col = turbine_mapping[f'electricity_{turbine}']
        turbine_name = mapping[synthetic_col]
        synth_turbine_power = data_a[synthetic_col].sum() / 1e6
        re_ninja_col = f'electricity_{turbine}'
        re_ninja_wind_col = f'wind_speed_{turbine}'
        synthetic_wind_col = f'wind_speed_t{turbine}'
        re_ninja_turbine_power = data_b[re_ninja_col].sum() / 1e6
        stats[f'synthetic {turbine_name}'] = synth_turbine_power
        stats[f're.ninja {turbine_name}'] = re_ninja_turbine_power
        diff_power = data_b[re_ninja_col] - data_a[synthetic_col]
        power_diffs[turbine_name] = diff_power
        diff_wind = data_b[re_ninja_wind_col] - data_a[synthetic_wind_col]
        wind_diffs[turbine_name] = diff_wind
        synth_sum += synth_turbine_power
        re_ninja_sum += re_ninja_turbine_power
        if print_stats:
            print(f'Turbine {turbine_name}:')
            print(f'Synthetic energy: {synth_turbine_power:.2f} GWh')
            print(f'Renewables.ninja energy: {re_ninja_turbine_power:.2f} GWh')
            print()
    if print_stats:
        print(f'Total synthetic energy: {synth_sum:.2f} GWh')
        print(f'Total renewables.ninja energy: {re_ninja_sum:.2f} GWh')
    return stats, power_diffs, wind_diffs

In [10]:
stats, power_diffs, wind_diffs = get_stats_and_diffs(data_a, data_b, mapping, turbine_mapping)

Turbine Enercon E-70 E4 2.300:
Synthetic energy: 72.01 GWh
Renewables.ninja energy: 127.12 GWh

Turbine Enercon E-82 E2 2.000:
Synthetic energy: 112.45 GWh
Renewables.ninja energy: 203.63 GWh

Turbine Enercon E-115 2.500:
Synthetic energy: 181.87 GWh
Renewables.ninja energy: 339.45 GWh

Turbine Vestas V90:
Synthetic energy: 104.75 GWh
Renewables.ninja energy: 194.64 GWh

Turbine Vestas V112-3.45:
Synthetic energy: 183.31 GWh
Renewables.ninja energy: 335.20 GWh

Turbine Vestas V80-1.8:
Synthetic energy: 75.66 GWh
Renewables.ninja energy: 144.74 GWh

Total synthetic energy: 730.05 GWh
Total renewables.ninja energy: 1344.78 GWh


In [26]:
data_a

,wind_speed,temperature_2m,relative_humidity,std_v_wind,pressure,wind_direction,w_vert,wind_speed_t1,temperature_t1,saturated_vapor_pressure,...,wind_speed_t5,temperature_t5,saturated_vapor_pressure_t5,density_t5,power_t5,wind_speed_t6,temperature_t6,saturated_vapor_pressure_t6,density_t6,power_t6
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00+00:00,2.883333,4.583333,90.033333,0.383333,997.700000,186.666667,0.000000,2.88,4.226383,847.542081,...,2.88,3.824003,803.543669,1.233356,0.000000,2.88,4.090093,818.726292,1.238415,0.000000
2024-01-01 01:00:00+00:00,3.100000,4.466667,89.050000,0.450000,997.700000,185.000000,0.000000,3.10,4.109717,840.646470,...,3.10,3.707337,796.965803,1.233936,0.000000,3.10,3.973427,812.038542,1.239000,0.000000
2024-01-01 02:00:00+00:00,2.433333,4.500000,88.016667,0.433333,997.516667,183.333333,0.000000,2.43,4.143050,842.611564,...,2.43,3.740670,798.840314,1.233595,0.000000,2.43,4.006760,813.944379,1.238656,0.000000
2024-01-01 03:00:00+00:00,2.416667,4.150000,88.983333,0.400000,997.483333,176.666667,0.000000,2.42,3.793050,822.179251,...,2.42,3.390670,779.351119,1.235140,0.000000,2.42,3.656760,794.129038,1.240214,0.000000
2024-01-01 04:00:00+00:00,1.933333,3.950000,89.000000,0.350000,997.400000,166.666667,0.000000,1.93,3.593050,810.701482,...,1.93,3.190670,768.404397,1.235964,0.000000,1.93,3.456760,782.998678,1.241046,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00+00:00,3.500000,-1.416667,93.716667,0.666667,1000.016667,248.333333,-0.018539,4.80,-1.773617,543.612603,...,5.49,-2.175997,510.217437,1.264413,371.522912,5.09,-1.909907,521.701011,1.269715,95.704416
2024-12-31 20:00:00+00:00,3.600000,-1.266667,92.550000,0.716667,999.966667,245.000000,-0.019077,4.94,-1.623617,550.440536,...,5.65,-2.025997,516.662291,1.263661,408.932630,5.23,-1.759907,528.277853,1.268956,110.282448
2024-12-31 21:00:00+00:00,3.366667,-1.300000,91.716667,0.733333,999.900000,256.666667,-0.020841,4.74,-1.656950,548.916501,...,5.48,-2.059330,515.223724,1.263758,369.040551,5.04,-1.793240,526.809838,1.269054,90.927562
